In [ ]:
from io import StringIO
from time import sleep

import pandas as pd
import openai
import networkx as nx
from pyvis.network import Network

In [ ]:
openai.api_key = 'Insert your API key here'
proteinNumber = 20
startGene = 'TNF'
context = 'macrophages'

In [ ]:
def getBinders(gene, context):
    prompt = f"What are up to " + str(proteinNumber) + " known proteins associated with " + gene + " in " + context + ", presented as a table with one column for Gene Symbol, and another columns with a description of how the protein is associated with " + gene + " in " + context + "?"
    print('Checking for ' + gene + ' in the context of ' + context)
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant that provides your answers in two-column tables and uses canonical gene symbols as identifiers for genes and proteins."},
            {"role": "user", "content": prompt}
        ]
    )

    message = response['choices'][0]['message']['content']
    message = '|'.join(message.split('|')[1:-1])
    
    #The try block is necessary just in case ChatGPT responds with a non-standard output
    try:
        df = pd.read_table(StringIO(message), sep='|')
        df.columns = ['Target', 'Association', 'd']
        df= df[['Target', 'Association']]
        df['Target'] = df['Target'].str.strip()
        df['Source'] = gene
        df = df.reset_index()
        del df['index']
        df.drop(0, inplace=True)
        return df
    except:
        print(message)
        return pd.DataFrame(columns= ['Target', 'Association', 'Source'])


In [ ]:
seedResult = getBinders(gene, context)
seedResult

In [ ]:
allResults = seedResult
for g in list(seedResult['Target']):
    result = getBinders(g.strip(), context)
    allResults = pd.concat([allResults, result])
    sleep(5) #This timeout is currently necessary for limited access accounts.  Feel free to disable if possible


In [ ]:
allResults.columns = ['target', 'title', 'source']
G  = nx.from_pandas_edgelist(allResults, source='source', target='target', edge_attr=True)
nt = Network(notebook=True)
nt.from_nx(G)

In [ ]:
nt.show('example.html')